In [11]:
%pip install webdavclient3

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'webdavclient3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'webdavclient3'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for webdavclient3: filename=webdavclient3-3.14.6-py3-none-any.whl size=20888 sha256=4814e06bf0bdab7925f6a375888c97da979f0a32afd9a058e5d349fcff89100b
  Stored in directory: /home/jovyan/.cache/pip/wheels/6e/14/66/e35c0618088019f3b11742283f7ecc5c89904eb27b96fe38d8
Successfully built webdavclient3
Note: you may need to restart the kernel to use updated packages.


In [1]:
from webdav3.client import Client
import getpass
password = getpass.getpass()

 ········


In [2]:
options = {
       'webdav_hostname': "https://webdav.hidrive.ionos.com/",
       'webdav_login': "xeliba",
       'webdav_password': password,
}

In [3]:
client = Client(options)

In [17]:
d = client.list("/users/xeliba", get_info=True)

In [18]:
d

[{'created': '2023-03-17T07:10:12Z',
  'name': None,
  'size': None,
  'modified': 'Thu, 31 Dec 2020 11:08:48 GMT',
  'etag': '"1d-5b7c0a5ded000"',
  'content_type': 'httpd/unix-directory',
  'isdir': True,
  'path': '/users/xeliba/BackupBis2016/'},
 {'created': '2023-03-17T10:44:17Z',
  'name': None,
  'size': None,
  'modified': 'Thu, 14 Apr 2022 20:25:07 GMT',
  'etag': '"2-5dca316fccec0"',
  'content_type': 'httpd/unix-directory',
  'isdir': True,
  'path': '/users/xeliba/Camera Upload/'},
 {'created': '2023-03-17T11:57:54Z',
  'name': None,
  'size': None,
  'modified': 'Mon, 30 Sep 2019 21:32:23 GMT',
  'etag': '"3-593cbf8a8a7c0"',
  'content_type': 'httpd/unix-directory',
  'isdir': True,
  'path': '/users/xeliba/Musik/'},
 {'created': '2023-03-17T08:49:27Z',
  'name': None,
  'size': None,
  'modified': 'Thu, 31 Dec 2020 06:54:15 GMT',
  'etag': '"9-5b7bd178753c0"',
  'content_type': 'httpd/unix-directory',
  'isdir': True,
  'path': '/users/xeliba/BilderBis2015/'},
 {'created'